In [ ]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    print("GPU available:", gpus)
else:
    print("No GPU available.")


No GPU available.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Step 2: Data Preparation
!unzip -q '/content/drive/MyDrive/IMG_CLASSES.zip' -d '/content/dataset'

In [ ]:
!scp -r /content/dataset '/content/drive/MyDrive'

In [ ]:
!pip install tensorflow


In [ ]:
!pip install onnx-tf


In [ ]:
import tensorflow as tf
from onnx_tf.backend import prepare


In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8l-cls.pt')  # load an official model
model = YOLO('/content/drive/MyDrive/YOLOALGO3Augmen/runs/classify/train/weights/best.pt')  # load a custom trained model

# Export the model
model.export(format='onnx')

Ultralytics YOLOv8.0.231 🚀 Python-3.10.12 torch-2.1.0+cu121 CPU (Intel Xeon 2.20GHz)
YOLOv8l-cls summary (fused): 133 layers, 36197386 parameters, 0 gradients, 98.7 GFLOPs

PyTorch: starting from '/content/drive/MyDrive/YOLOALGO3Augmen/runs/classify/train/weights/best.pt' with input shape (1, 3, 320, 320) BCHW and output shape(s) (1, 10) (69.2 MB)

ONNX: starting export with onnx 1.15.0 opset 17...
ONNX: export success ✅ 3.0s, saved as '/content/drive/MyDrive/YOLOALGO3Augmen/runs/classify/train/weights/best.onnx' (138.1 MB)

Export complete (7.4s)
Results saved to /content/drive/MyDrive/YOLOALGO3Augmen/runs/classify/train/weights
Predict:         yolo predict task=classify model=/content/drive/MyDrive/YOLOALGO3Augmen/runs/classify/train/weights/best.onnx imgsz=320  
Validate:        yolo val task=classify model=/content/drive/MyDrive/YOLOALGO3Augmen/runs/classify/train/weights/best.onnx imgsz=320 data=/content/dataset/IMG_CLASSES  
Visualize:       https://netron.app


'/content/drive/MyDrive/YOLOALGO3Augmen/runs/classify/train/weights/best.onnx'

In [ ]:
import onnx

# Load the ONNX model
onnx_model = onnx.load("/content/drive/MyDrive/YOLOALGO3Augmen/runs/classify/train/weights/best.onnx")  # Load the model object

# Prepare for TensorFlow conversion
tf_rep = prepare(onnx_model)


In [ ]:
tf_rep.export_graph("saved_model")


In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model("saved_model")
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # Enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS  # Enable TensorFlow ops.
]


In [ ]:
tflite_model = converter.convert()

# Save the TFLite model
open("converted_model.tflite", "wb").write(tflite_model)


144845368

In [ ]:
from sklearn.model_selection import train_test_split
import os
import shutil

# Define class names and the number of images per class
class_names = [
    "Atopic Dermatitis", "Basal Cell Carcinoma (BCC)", "Benign Keratosis-like Lesions (BKL)", "Eczema",
    "Melanocytic Nevi (NV)", "Melanoma", "Psoriasis pictures Lichen Planus and related diseases",
    "Seborrheic Keratoses and other Benign Tumors", "Tinea Ringworm Candidiasis and other Fungal Infections",
    "Warts Molluscum and other Viral Infections"
]

class_image_counts = [1989, 2495, 2079, 2504, 2527, 2516, 2512, 2363, 2498, 2103]


In [ ]:
# Organize the data into training and testing sets
base_dir = '/content/dataset/IMG_CLASSES'

# Create train and test directories
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

if not os.path.exists(train_dir):
    os.makedirs(train_dir)

if not os.path.exists(test_dir):
    os.makedirs(test_dir)

In [ ]:
# Move images into train and test directories
for class_name, image_count in zip(class_names, class_image_counts):
    class_dir = os.path.join(base_dir, class_name)
    images = os.listdir(class_dir)

In [ ]:
# Split the images into training and testing sets
    train_images, test_images = train_test_split(images, test_size=0.13, random_state=42)

    # Create class-specific directories in the train and test directories
    train_class_dir = os.path.join(train_dir, class_name)
    test_class_dir = os.path.join(test_dir, class_name)

    if not os.path.exists(train_class_dir):
        os.makedirs(train_class_dir)
    if not os.path.exists(test_class_dir):
        os.makedirs(test_class_dir)

In [ ]:
# Move images to their respective directories
    for image in train_images:
        src = os.path.join(class_dir, image)
        dst = os.path.join(train_class_dir, image)
        shutil.move(src, dst)

    for image in test_images:
        src = os.path.join(class_dir, image)
        dst = os.path.join(test_class_dir, image)
        shutil.move(src, dst)


In [ ]:
from PIL import Image
import os
# Function to resize images in a directory
def resize_images(directory, target_size):
    for root, dirs, files in os.walk(directory):
        for file in files:
            filepath = os.path.join(root, file)
            try:
                img = Image.open(filepath)
                img = img.resize(target_size, Image.ANTIALIAS)
                img.save(filepath)
            except Exception as e:
                print(f"Failed to process {filepath}: {e}")
target_size = (512, 512)
# Paths to train and test directories
train_directory = '/content/dataset/IMG_CLASSES/train'
test_directory = '/content/dataset/IMG_CLASSES/test'

# Resize images in train directory
resize_images(train_directory, target_size)

# Resize images in test directory
resize_images(test_directory, target_size)

<ipython-input-5-38ddd84c512d>:11: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize(target_size, Image.ANTIALIAS)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="whitegrid")
import os
import glob as gb
import cv2
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split


In [ ]:
import glob as gb

train_sizes = []  # List to store image sizes in the training dataset

# Iterate through each class in the training directory
for class_name in os.listdir(train_dir):
    train_class_dir = os.path.join(train_dir, class_name)

    # Iterate through each image in the class
    for file in gb.glob(pathname=os.path.join(train_class_dir, '*.jpg')):
        image = plt.imread(file)
        train_sizes.append(image.shape)

# Create a Pandas Series to count and display the unique image sizes
train_size_counts = pd.Series(train_sizes).value_counts()

# Print the counts of unique image sizes in the training dataset
print("Image sizes in the training dataset:")
print(train_size_counts)


Image sizes in the training dataset:
(512, 512, 3)    17774
dtype: int64


In [ ]:
test_sizes = []  # List to store image sizes in the training dataset

# Iterate through each class in the training directory
for class_name in os.listdir(test_dir):
    test_class_dir = os.path.join(test_dir, class_name)

    # Iterate through each image in the class
    for file in gb.glob(pathname=os.path.join(test_class_dir, '*.jpg')):
        image = plt.imread(file)
        test_sizes.append(image.shape)

# Create a Pandas Series to count and display the unique image sizes
test_size_counts = pd.Series(test_sizes).value_counts()

# Print the counts of unique image sizes in the training dataset
print("Image sizes in the testing dataset:")
print(test_size_counts)


Image sizes in the testing dataset:
(512, 512, 3)    3144
dtype: int64


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
# Create an ImageDataGenerator with augmentation configurations
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Iterate through each class in the training directory
for class_name in class_names:
    class_dir = os.path.join(train_dir, class_name)
    images = os.listdir(class_dir)

In [ ]:
    # Apply augmentation to each image in the class directory
    for image_name in images:
        image_path = os.path.join(class_dir, image_name)
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.reshape((1,) + img.shape)  # Reshape for flow method
        save_prefix = 'augmented_' + image_name.split('.')[0]  # Prefix for augmented images

        # Generate augmented images
        i = 0
        for batch in datagen.flow(img, batch_size=1, save_to_dir=class_dir, save_prefix=save_prefix, save_format='jpg'):
            i += 1
            if i >= 3:  # Generate 3 augmented versions of each image
                break

In [ ]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()
from ultralytics import YOLO

from IPython.display import display, Image

Ultralytics YOLOv8.0.231 🚀 Python-3.10.12 torch-2.1.0+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.4/225.8 GB disk)


In [ ]:
from ultralytics import YOLO
DATA_DIR = '/content/dataset/IMG_CLASSES'
# Load a model
model = YOLO('yolov8x-cls.yaml')  # build a new model from YAML
model = YOLO('yolov8x-cls.pt')  # load a pretrained model (recommended for training)
model = YOLO('yolov8x-cls.yaml').load('yolov8x-cls.pt')  # build from YAML and transfer weights

# Train the model
results = model.train(data=DATA_DIR, epochs=32, imgsz=416, lr0=0.001, resume=True)



                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1   1844480  ultralytics.nn.modules.conv.Conv             [320, 640, 3, 2]              
  6                  -1  6  13117440  ultralytics.nn.modules.block.C2f             [640, 640, 6, True]           
  7                  -1  1   7375360  ultralytics.nn.modules.conv.Conv             [640

train: Scanning /content/dataset/IMG_CLASSES/train... 72764 images, 0 corrupt: 100%|██████████| 72764/72764 [00:00<?, ?it/s]


albumentations: RandomResizedCrop(p=1.0, height=416, width=416, scale=(0.5, 1.0), ratio=(0.75, 1.3333333333333333), interpolation=1), HorizontalFlip(p=0.5), ColorJitter(p=0.5, brightness=[0.6, 1.4], contrast=[0.6, 1.4], saturation=[0.30000000000000004, 1.7], hue=[-0.015, 0.015]), Normalize(p=1.0, mean=(0.0, 0.0, 0.0), std=(1.0, 1.0, 1.0), max_pixel_value=255.0), ToTensorV2(always_apply=True, p=1.0, transpose_mask=False)


val: Scanning /content/dataset/IMG_CLASSES/test... 2726 images, 0 corrupt: 100%|██████████| 2726/2726 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.001' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 50 weight(decay=0.0), 51 weight(decay=0.0005), 51 bias(decay=0.0)
32 epochs...

      Epoch    GPU_mem       loss  Instances       Size


...


      Epoch    GPU_mem       loss  Instances       Size


      16/32      5.71G     0.1684         12        416: 100%|██████████| 4548/4548 [25:47<00:00,  2.94it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 86/86 [00:18<00:00,  4.70it/s]

                   all      0.868      0.995



      Epoch    GPU_mem       loss  Instances       Size


      17/32      5.77G     0.1176         16        416:   0%|          | 19/4548 [00:06<25:38,  2.94it/s]

In [ ]:
from ultralytics import YOLO
DATA_DIR = '/content/dataset/IMG_CLASSES'
# Load a model
model = YOLO('yolov8l-cls.yaml')  # build a new model from YAML
model = YOLO('yolov8l-cls.pt')  # load a pretrained model (recommended for training)
model = YOLO('yolov8l-cls.yaml').load('yolov8l-cls.pt')  # build from YAML and transfer weights

# Train the model
results = model.train(data= DATA_DIR, epochs=32, imgsz=320)


                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  3    279808  ultralytics.nn.modules.block.C2f             [128, 128, 3, True]           
  3                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  4                  -1  6   2101248  ultralytics.nn.modules.block.C2f             [256, 256, 6, True]           
  5                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256, 512, 3, 2]              
  6                  -1  6   8396800  ultralytics.nn.modules.block.C2f             [512, 512, 6, True]           
  7                  -1  1   4720640  ultralytics.nn.modules.conv.Conv             [512

train: Scanning /content/dataset/IMG_CLASSES/train... 71092 images, 0 corrupt: 100%|██████████| 71092/71092 [00:00<?, ?it/s]


albumentations: RandomResizedCrop(p=1.0, height=320, width=320, scale=(0.5, 1.0), ratio=(0.75, 1.3333333333333333), interpolation=1), HorizontalFlip(p=0.5), ColorJitter(p=0.5, brightness=[0.6, 1.4], contrast=[0.6, 1.4], saturation=[0.30000000000000004, 1.7], hue=[-0.015, 0.015]), Normalize(p=1.0, mean=(0.0, 0.0, 0.0), std=(1.0, 1.0, 1.0), max_pixel_value=255.0), ToTensorV2(always_apply=True, p=1.0, transpose_mask=False)


val: Scanning /content/dataset/IMG_CLASSES/test... 3144 images, 0 corrupt: 100%|██████████| 3144/3144 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 50 weight(decay=0.0), 51 weight(decay=0.0005), 51 bias(decay=0.0)
32 epochs...

      Epoch    GPU_mem       loss  Instances       Size


...


      Epoch    GPU_mem       loss  Instances       Size


      32/32       3.1G    0.01842          4        320: 100%|██████████| 4444/4444 [09:46<00:00,  7.58it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 99/99 [00:08<00:00, 11.61it/s]


                   all      0.853      0.994

32 epochs completed in 5.326 hours.
Optimizer stripped from runs/classify/train2/weights/last.pt, 72.6MB
Optimizer stripped from runs/classify/train2/weights/best.pt, 72.6MB

Validating runs/classify/train2/weights/best.pt...
Ultralytics YOLOv8.0.229 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8l-cls summary (fused): 133 layers, 36197386 parameters, 0 gradients, 98.7 GFLOPs
WARNING ⚠️ Dataset 'split=val' not found, using 'split=test' instead.
train: /content/dataset/IMG_CLASSES/train... found 71092 images in 10 classes ✅ 
val: None...
test: /content/dataset/IMG_CLASSES/test... found 3144 images in 10 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 99/99 [00:09<00:00, 10.93it/s]


                   all      0.854      0.996
Speed: 0.1ms preprocess, 2.6ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train2
Results saved to runs/classify/train2


In [ ]:
from ultralytics import YOLO
DATA_DIR = '/content/dataset/IMG_CLASSES'
# Load a model
model = YOLO('yolov8l-cls.yaml')  # build a new model from YAML
model = YOLO('yolov8l-cls.pt')  # load a pretrained model (recommended for training)
model = YOLO('yolov8l-cls.yaml').load('yolov8l-cls.pt')  # build from YAML and transfer weights

# Train the model
results = model.train(data= DATA_DIR, epochs=30, imgsz=320)


                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  3    279808  ultralytics.nn.modules.block.C2f             [128, 128, 3, True]           
  3                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  4                  -1  6   2101248  ultralytics.nn.modules.block.C2f             [256, 256, 6, True]           
  5                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256, 512, 3, 2]              
  6                  -1  6   8396800  ultralytics.nn.modules.block.C2f             [512, 512, 6, True]           
  7                  -1  1   4720640  ultralytics.nn.modules.conv.Conv             [512

100%|██████████| 71.7M/71.7M [00:00<00:00, 453MB/s]



                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  3    279808  ultralytics.nn.modules.block.C2f             [128, 128, 3, True]           
  3                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  4                  -1  6   2101248  ultralytics.nn.modules.block.C2f             [256, 256, 6, True]           
  5                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256, 512, 3, 2]              
  6                  -1  6   8396800  ultralytics.nn.modules.block.C2f             [512, 512, 6, True]           
  7                  -1  1   4720640  ultralytics.nn.modules.conv.Conv             [512

100%|██████████| 6.23M/6.23M [00:00<00:00, 298MB/s]


AMP: checks passed ✅


...


      Epoch    GPU_mem       loss  Instances       Size


      30/30      2.46G    0.01982          6        320: 100%|██████████| 4444/4444 [09:21<00:00,  7.92it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 99/99 [00:08<00:00, 12.25it/s]


                   all      0.867      0.997

30 epochs completed in 4.772 hours.
Optimizer stripped from runs/classify/train/weights/last.pt, 72.6MB
Optimizer stripped from runs/classify/train/weights/best.pt, 72.6MB

Validating runs/classify/train/weights/best.pt...
Ultralytics YOLOv8.0.228 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8l-cls summary (fused): 133 layers, 36197386 parameters, 0 gradients, 98.7 GFLOPs
WARNING ⚠️ Dataset 'split=val' not found, using 'split=test' instead.
train: /content/dataset/IMG_CLASSES/train... found 71094 images in 10 classes ✅ 
val: None...
test: /content/dataset/IMG_CLASSES/test... found 3144 images in 10 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 99/99 [00:08<00:00, 11.44it/s]


                   all      0.869      0.997
Speed: 0.2ms preprocess, 2.5ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train
Results saved to runs/classify/train


In [ ]:
!scp -r /content/converted_model.tflite '/content/drive/MyDrive/YOLOTF1'

In [ ]:
import os

from ultralytics import YOLO

DATA_DIR = '/content/dataset/IMG_CLASSES'
# Load a model
model = YOLO("yolov8l-cls.pt")  # load a pretained modelll

# Use the model
results = model.train(data=DATA_DIR, epochs=30, imgsz=640)  # train the model


100%|██████████| 71.7M/71.7M [00:00<00:00, 408MB/s]


Ultralytics YOLOv8.0.226 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8l-cls.pt, data=/content/dataset/IMG_CLASSES, epochs=30, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, forma

train: Scanning /content/dataset/IMG_CLASSES/train... 16730 images, 0 corrupt: 100%|██████████| 16730/16730 [00:05<00:00, 3038.42it/s]


train: New cache created: /content/dataset/IMG_CLASSES/train.cache
albumentations: RandomResizedCrop(p=1.0, height=640, width=640, scale=(0.5, 1.0), ratio=(0.75, 1.3333333333333333), interpolation=1), HorizontalFlip(p=0.5), ColorJitter(p=0.5, brightness=[0.6, 1.4], contrast=[0.6, 1.4], saturation=[0.30000000000000004, 1.7], hue=[-0.015, 0.015]), Normalize(p=1.0, mean=(0.0, 0.0, 0.0), std=(1.0, 1.0, 1.0), max_pixel_value=255.0), ToTensorV2(always_apply=True, p=1.0, transpose_mask=False)


val: Scanning /content/dataset/IMG_CLASSES/val... 4188 images, 0 corrupt: 100%|██████████| 4188/4188 [00:01<00:00, 3719.09it/s]


...


      Epoch    GPU_mem       loss  Instances       Size


               classes   top1_acc   top5_acc: 100%|██████████| 131/131 [00:44<00:00,  2.96it/s]

                   all       0.76      0.992



      Epoch    GPU_mem       loss  Instances       Size


       9/30      7.82G     0.6029         16        640:  35%|███▍      | 366/1046 [03:03<05:41,  1.99it/s]

In [ ]:
import os

from ultralytics import YOLO

DATA_DIR = '/content/dataset/IMG_CLASSES'
# Load a model
model = YOLO("yolov8l-cls.pt")  # load a pretained model

# Use the model
results = model.train(data=DATA_DIR, epochs=30, imgsz=250)  # train the model


Ultralytics YOLOv8.0.225 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8l-cls.pt, data=/content/dataset/IMG_CLASSES, epochs=30, patience=50, batch=16, imgsz=250, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, form

train: Scanning /content/dataset/IMG_CLASSES/train... 16730 images, 0 corrupt: 100%|██████████| 16730/16730 [00:00<?, ?it/s]


albumentations: RandomResizedCrop(p=1.0, height=256, width=256, scale=(0.5, 1.0), ratio=(0.75, 1.3333333333333333), interpolation=1), HorizontalFlip(p=0.5), ColorJitter(p=0.5, brightness=[0.6, 1.4], contrast=[0.6, 1.4], saturation=[0.30000000000000004, 1.7], hue=[-0.015, 0.015]), Normalize(p=1.0, mean=(0.0, 0.0, 0.0), std=(1.0, 1.0, 1.0), max_pixel_value=255.0), ToTensorV2(always_apply=True, p=1.0, transpose_mask=False)


val: Scanning /content/dataset/IMG_CLASSES/val... 4188 images, 0 corrupt: 100%|██████████| 4188/4188 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 50 weight(decay=0.0), 51 weight(decay=0.0005), 51 bias(decay=0.0)
Image sizes 256 train, 256 val
Using 8 dataloader workers
Logging results to runs/classify/train4
Starting training for 30 epochs...

      Epoch    GPU_mem       loss  Instances       Size


...

      30/30      2.04G    0.05675         10        256: 100%|██████████| 1046/1046 [01:33<00:00, 11.21it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 131/131 [00:08<00:00, 14.87it/s]

                   all      0.845      0.995



30 epochs completed in 0.872 hours.
Optimizer stripped from runs/classify/train4/weights/last.pt, 72.6MB
Optimizer stripped from runs/classify/train4/weights/best.pt, 72.6MB

Validating runs/classify/train4/weights/best.pt...
Ultralytics YOLOv8.0.225 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8l-cls summary (fused): 133 layers, 36197386 parameters, 0 gradients, 98.7 GFLOPs
train: /content/dataset/IMG_CLASSES/train... found 16730 images in 10 classes ✅ 
val: /content/dataset/IMG_CLASSES/val... found 4188 images in 10 classes ✅ 
test: None...


               classes   top1_acc   top5_acc: 100%|██████████| 131/131 [00:09<00:00, 14.14it/s]


                   all      0.847      0.994
Speed: 0.1ms preprocess, 1.7ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train4
Results saved to runs/classify/train4


In [ ]:
import os

from ultralytics import YOLO

DATA_DIR = '/content/dataset/IMG_CLASSES'
# Load a model
model = YOLO("yolov8l-cls.pt")  # load a pretained model

# Use the model
results = model.train(data=DATA_DIR, epochs=50, imgsz=224)  # train the model


Ultralytics YOLOv8.0.225 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8l-cls.pt, data=/content/dataset/IMG_CLASSES, epochs=50, patience=50, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, form

train: Scanning /content/dataset/IMG_CLASSES/train... 16730 images, 0 corrupt: 100%|██████████| 16730/16730 [00:00<?, ?it/s]

albumentations: RandomResizedCrop(p=1.0, height=224, width=224, scale=(0.5, 1.0), ratio=(0.75, 1.3333333333333333), interpolation=1), HorizontalFlip(p=0.5), ColorJitter(p=0.5, brightness=[0.6, 1.4], contrast=[0.6, 1.4], saturation=[0.30000000000000004, 1.7], hue=[-0.015, 0.015]), Normalize(p=1.0, mean=(0.0, 0.0, 0.0), std=(1.0, 1.0, 1.0), max_pixel_value=255.0), ToTensorV2(always_apply=True, p=1.0, transpose_mask=False)



val: Scanning /content/dataset/IMG_CLASSES/val... 4188 images, 0 corrupt: 100%|██████████| 4188/4188 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 50 weight(decay=0.0), 51 weight(decay=0.0005), 51 bias(decay=0.0)
Image sizes 224 train, 224 val
Using 8 dataloader workers
Logging results to runs/classify/train5
Starting training for 50 epochs...

      Epoch    GPU_mem       loss  Instances       Size


...

      50/50       1.5G    0.02416         10        224: 100%|██████████| 1046/1046 [01:18<00:00, 13.26it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 131/131 [00:08<00:00, 14.93it/s]

                   all      0.842      0.991



50 epochs completed in 1.256 hours.
Optimizer stripped from runs/classify/train5/weights/last.pt, 72.6MB
Optimizer stripped from runs/classify/train5/weights/best.pt, 72.6MB

Validating runs/classify/train5/weights/best.pt...
Ultralytics YOLOv8.0.225 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8l-cls summary (fused): 133 layers, 36197386 parameters, 0 gradients, 98.7 GFLOPs
train: /content/dataset/IMG_CLASSES/train... found 16730 images in 10 classes ✅ 
val: /content/dataset/IMG_CLASSES/val... found 4188 images in 10 classes ✅ 
test: None...


               classes   top1_acc   top5_acc: 100%|██████████| 131/131 [00:08<00:00, 14.87it/s]


                   all      0.843      0.993
Speed: 0.1ms preprocess, 1.4ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train5
Results saved to runs/classify/train5


In [ ]:
import os

from ultralytics import YOLO

DATA_DIR = '/content/dataset/IMG_CLASSES'
# Load a model
model = YOLO("yolov8x-cls.pt")  # load a pretained model

# Use the model
results = model.train(data=DATA_DIR, epochs=30, imgsz=300)  # train the model


In [ ]:
!scp -r /content/runs '/content/drive/MyDrive/YOLOALGO1'

In [ ]:
from ultralytics import YOLO
DATA_DIR = '/content/dataset/IMG_CLASSES'

model = YOLO("yolov8l-cls.pt")


# Train the model
results = model.train(data=DATA_DIR, epochs=50, imgsz=512)

100%|██████████| 71.7M/71.7M [00:00<00:00, 462MB/s]


Ultralytics YOLOv8.0.225 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8l-cls.pt, data=/content/dataset/IMG_CLASSES, epochs=50, patience=50, batch=16, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, form

train: Scanning /content/dataset/IMG_CLASSES/train... 16730 images, 0 corrupt: 100%|██████████| 16730/16730 [00:00<?, ?it/s]

albumentations: RandomResizedCrop(p=1.0, height=512, width=512, scale=(0.5, 1.0), ratio=(0.75, 1.3333333333333333), interpolation=1), HorizontalFlip(p=0.5), ColorJitter(p=0.5, brightness=[0.6, 1.4], contrast=[0.6, 1.4], saturation=[0.30000000000000004, 1.7], hue=[-0.015, 0.015]), Normalize(p=1.0, mean=(0.0, 0.0, 0.0), std=(1.0, 1.0, 1.0), max_pixel_value=255.0), ToTensorV2(always_apply=True, p=1.0, transpose_mask=False)



val: Scanning /content/dataset/IMG_CLASSES/val... 4188 images, 0 corrupt: 100%|██████████| 4188/4188 [00:00<?, ?it/s]


...

      50/50      6.43G    0.01976         10        512: 100%|██████████| 1046/1046 [05:09<00:00,  3.37it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 131/131 [00:26<00:00,  4.94it/s]

                   all      0.861      0.995



50 epochs completed in 4.698 hours.
Optimizer stripped from runs/classify/train8/weights/last.pt, 72.6MB
Optimizer stripped from runs/classify/train8/weights/best.pt, 72.6MB

Validating runs/classify/train8/weights/best.pt...
Ultralytics YOLOv8.0.225 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8l-cls summary (fused): 133 layers, 36197386 parameters, 0 gradients, 98.7 GFLOPs
train: /content/dataset/IMG_CLASSES/train... found 16730 images in 10 classes ✅ 
val: /content/dataset/IMG_CLASSES/val... found 4188 images in 10 classes ✅ 
test: None...


               classes   top1_acc   top5_acc: 100%|██████████| 131/131 [00:27<00:00,  4.78it/s]


                   all      0.866      0.995
Speed: 0.3ms preprocess, 5.9ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train8
Results saved to runs/classify/train8


In [ ]:
import os

from ultralytics import YOLO

DATA_DIR = '/content/dataset/IMG_CLASSES'
# Load a model
model = YOLO("yolov8l-cls.pt")  # load a pretained modellll

# Use the model
results = model.train(data=DATA_DIR, epochs=40, imgsz=512)  # train the model


Ultralytics YOLOv8.0.227 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8l-cls.pt, data=/content/dataset/IMG_CLASSES, epochs=40, patience=50, batch=16, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, form

train: Scanning /content/dataset/IMG_CLASSES/train... 17774 images, 0 corrupt: 100%|██████████| 17774/17774 [00:06<00:00, 2817.93it/s]


train: New cache created: /content/dataset/IMG_CLASSES/train.cache
albumentations: RandomResizedCrop(p=1.0, height=512, width=512, scale=(0.5, 1.0), ratio=(0.75, 1.3333333333333333), interpolation=1), HorizontalFlip(p=0.5), ColorJitter(p=0.5, brightness=[0.6, 1.4], contrast=[0.6, 1.4], saturation=[0.30000000000000004, 1.7], hue=[-0.015, 0.015]), Normalize(p=1.0, mean=(0.0, 0.0, 0.0), std=(1.0, 1.0, 1.0), max_pixel_value=255.0), ToTensorV2(always_apply=True, p=1.0, transpose_mask=False)


val: Scanning /content/dataset/IMG_CLASSES/test... 3144 images, 0 corrupt: 100%|██████████| 3144/3144 [00:00<00:00, 3184.41it/s]

val: New cache created: /content/dataset/IMG_CLASSES/test.cache


...

      40/40      5.15G    0.02203         14        512: 100%|██████████| 1111/1111 [05:20<00:00,  3.47it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 99/99 [00:19<00:00,  4.98it/s]

                   all      0.868      0.994



40 epochs completed in 3.800 hours.
Optimizer stripped from runs/classify/train2/weights/last.pt, 72.6MB
Optimizer stripped from runs/classify/train2/weights/best.pt, 72.6MB

Validating runs/classify/train2/weights/best.pt...
Ultralytics YOLOv8.0.227 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8l-cls summary (fused): 133 layers, 36197386 parameters, 0 gradients, 98.7 GFLOPs
WARNING ⚠️ Dataset 'split=val' not found, using 'split=test' instead.
train: /content/dataset/IMG_CLASSES/train... found 17774 images in 10 classes ✅ 
val: None...
test: /content/dataset/IMG_CLASSES/test... found 3144 images in 10 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 99/99 [00:20<00:00,  4.77it/s]


                   all       0.87      0.994
Speed: 0.3ms preprocess, 5.7ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train2
Results saved to runs/classify/train2


In [ ]:
!scp -r /content/runs '/content/drive/MyDrive/YOLOALGO2'

In [ ]:
model = YOLO("yolov8l-cls.pt")

In [ ]:
# Test directory
test_dir = os.path.join(base_dir, 'test')

In [ ]:
from ultralytics import YOLO

# Load the trained model using the best.pt checkpoint
model = YOLO('/content/drive/MyDrive/YOLOALGO2/runs/classify/train2/weights/best.pt')

# Validate the model
metrics = model.val()

# Display validation results
print("Top-1 Accuracy:", metrics.top1)
print("Top-5 Accuracy:", metrics.top5)


Ultralytics YOLOv8.0.227 🚀 Python-3.10.12 torch-2.1.0+cu118 CPU (Intel Xeon 2.20GHz)
YOLOv8l-cls summary (fused): 133 layers, 36197386 parameters, 0 gradients, 98.7 GFLOPs
WARNING ⚠️ Dataset 'split=val' not found, using 'split=test' instead.
train: /content/dataset/IMG_CLASSES/train... found 17774 images in 10 classes ✅ 
val: None...
test: /content/dataset/IMG_CLASSES/test... found 3144 images in 10 classes ✅ 


TypeError: ignored